
# Vidhi – Demo Workflow (Final)

This notebook demonstrates the **end-to-end Vidhi multi-agent workflow** using:

- LangGraph-based orchestration
- Agent-level tracing (`logs/agent_traces.log`)
- Evaluation hooks aligned with `tests/`
- Explicit human-in-the-loop enforcement

⚠️ **All outputs are drafts and must be reviewed by a qualified lawyer.**



## 1. Setup & Imports


In [ ]:

from uuid import uuid4
from src.graph.graph import build_vidhi_graph



## 2. Provide Case Inputs


In [ ]:

case_facts = {
    "facts": "Tenant eviction dispute due to alleged non-payment of rent",
    "timeline": {
        "notice_date": "2022-01-10",
        "eviction_filed": "2023-03-01"
    }
}

jurisdiction = "Maharashtra High Court"
case_type = "Civil"



## 3. Build Initial Graph State


In [ ]:

initial_state = {
    "session_id": str(uuid4()),

    "case_facts": case_facts,
    "jurisdiction": jurisdiction,
    "case_type": case_type,

    # Agent outputs (placeholders)
    "precedents": [],
    "issues": [],
    "limitation_analysis": {},
    "arguments": {},
    "draft_document": "",
    "compliance_flags": [],
    "legal_aid_options": [],

    # Observability
    "evaluation": {},
    "human_review_required": False,
}



## 4. Execute Vidhi LangGraph


In [ ]:

graph = build_vidhi_graph()
final_state = graph.invoke(initial_state)



## 5. Inspect Key Outputs


In [ ]:

print("Issues Identified:")
print(final_state["issues"])

print("\nLimitation Analysis:")
print(final_state["limitation_analysis"])

print("\nCompliance Flags:")
print(final_state["compliance_flags"])

print("\nHuman Review Required:", final_state["human_review_required"])



## 6. Human-in-the-Loop (Mandatory)

At this stage:
- No document should be filed
- No legal opinion should be relied upon
- Outputs are **research drafts only**

Human validation is required before proceeding.



## 7. Evaluation Hooks (Aligned with tests/)


In [ ]:

# Example evaluation signals
evaluation = {
    "precedent_coverage": len(final_state["precedents"]) > 0,
    "issues_identified": len(final_state["issues"]) >= 1,
    "limitation_checked": bool(final_state["limitation_analysis"),
    "no_fabrication": "supreme court" not in final_state["draft_document"].lower(),
}

final_state["evaluation"] = evaluation
evaluation



## 8. Safety & Governance Assertions


In [ ]:

assert final_state["human_review_required"] in [True, False]
assert "fabricated" not in final_state["draft_document"].lower()



## 9. What This Notebook Demonstrates

- Clean separation of **logic vs orchestration**
- Production-aligned LangGraph usage
- Traceable, testable, governable agent workflow
- Explicit safety boundaries

This notebook mirrors how Vidhi would run in API or UI mode.
